In [2]:
import pandas as pd
import plotly.express as px

#I load the dataset
file_path = '/Users/michelangelogasola/Desktop/estimated-cumulative-excess-deaths-per-100000-people-during-covid-19.csv'
data = pd.read_csv(file_path)

#I select relevant columns
data_cleaned = data[['Entity', 'Day', 'Cumulative excess deaths per 100,000 people (central estimate)']]
data_cleaned.columns = ['Country', 'Date', 'Excess_Deaths']

#I drop rows with missing data
data_cleaned = data_cleaned.dropna(subset=['Excess_Deaths'])

#I format 'Date' column and aggregate to monthly
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'])
data_cleaned['Month'] = data_cleaned['Date'].dt.to_period('M')  # Aggregate to monthly periods
data_monthly = data_cleaned.groupby(['Country', 'Month'], as_index=False)['Excess_Deaths'].mean()

#I convert 'Month' back to string for Plotly animation
data_monthly['Month'] = data_monthly['Month'].dt.strftime('%Y-%m')

#Ccolor scheme
final_color_scale = [
    [0, "rgb(240,249,255)"],  # Light blue
    [0.2, "rgb(255,247,188)"],  # Light yellow
    [0.4, "rgb(254,227,145)"],  # Yellow
    [0.6, "rgb(254,196,79)"],   # Dark yellow
    [0.8, "rgb(252,141,89)"],   # Light red
    [0.9, "rgb(227,26,28)"],    # Red
    [1.0, "rgb(153,0,13)"]      # Dark red
]

#I create the choropleth map
final_fig = px.choropleth(
    data_monthly,
    locations="Country",
    locationmode="country names",
    color="Excess_Deaths",
    hover_name="Country",
    animation_frame="Month",
    title="Cumulative Excess Deaths per 100,000 People (Monthly)",
    color_continuous_scale=final_color_scale,
    range_color=(0, 800)  # Fixed legend range
)

#I update layout for better appearance
final_fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type="natural earth"
    ),
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    sliders=[
        dict(
            currentvalue={"visible": True, "prefix": "Month: "},
            transition={"duration": 50},  # Faster transitions
            pad={"t": 50}
        )
    ]
)

#I show the final visualization
final_fig.show()
